In [9]:
import sys
sys.path.append("../../helios-server/")
import helios.models as models
from helios_auth import models as auth_model
import helios.views as views
from helios.models import CastVote, Voter, Trustee
from helios.workflows.homomorphic import EncryptedVote, Tally, EncryptedAnswer
from helios.crypto import algs
from election_utils import *
from tqdm import tqdm, trange
# import click
import time
import os
import matplotlib.pyplot as plt
import numpy as np
from itertools import combinations
import pickle

In [3]:
# num_questions = 4
num_choices = 4
choices = [i for i in range(num_choices)]

In [4]:
possible_answers = []
for i in trange(num_choices+1):
    possible_answers.extend([list(xs) for xs in combinations(choices, i)])

100%|██████████| 5/5 [00:00<00:00, 6773.75it/s]


In [5]:
possible_answers

[[],
 [0],
 [1],
 [2],
 [3],
 [0, 1],
 [0, 2],
 [0, 3],
 [1, 2],
 [1, 3],
 [2, 3],
 [0, 1, 2],
 [0, 1, 3],
 [0, 2, 3],
 [1, 2, 3],
 [0, 1, 2, 3]]

In [6]:
election_keypair = pickle.load(open("election_keypair.p", "rb"))

In [15]:
def fromElectionAndAnswer(choice_indicies, answer, pk, q_max, q_min=0):
    """
    Given an election, a question number, and a list of answers to that question
    in the form of an array of 0-based indexes into the answer array,
    produce an EncryptedAnswer that works.
    """
    
    # initialize choices, individual proofs, randomness and overall proof
    choices = [None for a in range(len(choice_indicies))]
    individual_proofs = [None for a in range(len(choice_indicies))]
    overall_proof = None
    randomness = [None for a in range(len(choice_indicies))]
    
    # possible plaintexts [0, 1]
    plaintexts = EncryptedAnswer.generate_plaintexts(pk)
    
    # keep track of number of options selected.
    num_selected_answers = 0
    
    # homomorphic sum of all
    homomorphic_sum = 0
    randomness_sum = 0

    # min and max for number of answers, useful later

    min_answers = q_min
    max_answers = q_max

    # go through each possible answer and encrypt either a g^0 or a g^1.
    for answer_num in range(len(choice_indicies)):
      plaintext_index = 0
      
      # assuming a list of answers
      if answer_num in answer:
        plaintext_index = 1
        num_selected_answers += 1

      # randomness and encryption
      randomness[answer_num] = algs.random.mpz_lt(pk.q)
      choices[answer_num] = pk.encrypt_with_r(plaintexts[plaintext_index], randomness[answer_num])
      
      # generate proof
      individual_proofs[answer_num] = choices[answer_num].generate_disjunctive_encryption_proof(plaintexts, plaintext_index, 
                                                randomness[answer_num], algs.EG_disjunctive_challenge_generator)
                                                
      # sum things up homomorphically if needed
      if max_answers is not None:
        homomorphic_sum = choices[answer_num] * homomorphic_sum
        randomness_sum = (randomness_sum + randomness[answer_num]) % pk.q

    # prove that the sum is 0 or 1 (can be "blank vote" for this answer)
    # num_selected_answers is 0 or 1, which is the index into the plaintext that is actually encoded
    
    if num_selected_answers < min_answers:
      raise Exception("Need to select at least %s answer(s)" % min_answers)
    
    if max_answers is not None:
      sum_plaintexts = EncryptedAnswer.generate_plaintexts(pk, min=min_answers, max=max_answers)
    
      # need to subtract the min from the offset
      overall_proof = homomorphic_sum.generate_disjunctive_encryption_proof(sum_plaintexts, num_selected_answers - min_answers, randomness_sum, algs.EG_disjunctive_challenge_generator);
    else:
      # approval voting
      overall_proof = None
    
    return EncryptedAnswer(choices, individual_proofs, overall_proof, randomness, answer)

In [28]:
for num_choices in range(1, 5):
    choices = [i for i in range(num_choices)]
    print(f"Enumerating possible answers with {num_choices} choices")
    possible_answers = []
    for i in trange(num_choices+1):
        possible_answers.extend([list(xs) for xs in combinations(choices, i)])
    
    print(f"Generation encyptions")
    encypted_answers = [fromElectionAndAnswer(choices, answer, election_keypair["public_key"], num_choices) for answer in tqdm(possible_answers)]
    np.save(f"encrypted_answers/all_answers_with_{num_choices}_choices.npy", encypted_answers,  allow_pickle=True)

Enumerating possible answers with 1 choices


100%|██████████| 2/2 [00:00<00:00, 9788.34it/s]


Generation encyptions


100%|██████████| 2/2 [00:00<00:00,  5.80it/s]


Enumerating possible answers with 2 choices


...

100%|██████████| 8/8 [00:03<00:00,  2.22it/s]


Enumerating possible answers with 4 choices


100%|██████████| 5/5 [00:00<00:00, 14246.96it/s]


Generation encyptions


100%|██████████| 16/16 [00:08<00:00,  1.85it/s]


In [27]:
list(range(1, 5))

[1, 2, 3, 4]

In [20]:
xs = np.load(f"encrypted_answers/all_answers_with_{num_choices}_choices.npy",  allow_pickle=True)